# 라이브러리 임포트 및 드라이브 마운트

In [ ]:
!git clone https://github.com/JunkyByte/easy_ViTPose.git
%cd easy_ViTPose
!pip install -r requirements.txt
!pip install -e .
!pip install huggingface_hub
!pip install onnx onnxruntime-gpu

Cloning into 'easy_ViTPose'...
remote: Enumerating objects: 682, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 682 (delta 55), reused 62 (delta 26), pack-reused 583
Receiving objects: 100% (682/682), 8.71 MiB | 9.40 MiB/s, done.
Resolving deltas: 100% (392/392), done.
/content/easy_ViTPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 80.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.5/609.5 kB 28.3 M

Obtaining file:///content/easy_ViTPose
  Preparing metadata (setup.py) ... done
  Running setup.py develop for easy-ViTPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 76.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import os
from google.colab import files

from huggingface_hub import hf_hub_download

import numpy as np
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# ViTPose main_code edit

In [ ]:
MODEL_TYPE = 'onnx'
MODEL_SIZE = 'l'
YOLO_TYPE = 'onnx'
YOLO_SIZE = 'n'
ext = {'tensorrt': '.engine', 'onnx': '.onnx', 'torch': '.pth'}[MODEL_TYPE]
ext_yolo = {'onnx': '.onnx', 'torch': '.pt'}[YOLO_TYPE]

REPO_ID = 'JunkyByte/easy_ViTPose'
FILENAME = os.path.join(MODEL_TYPE, 'vitpose-25-' + MODEL_SIZE) + ext
FILENAME_YOLO = 'yolov5/yolov5' + YOLO_SIZE + ext_yolo

print(f'Downloading model {REPO_ID}/{FILENAME}')
model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
yolo_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME_YOLO)

In [ ]:
%cd /content/easy_ViTPose/easy_ViTPose/
!ls

/content/easy_ViTPose/easy_ViTPose
configs      easy_ViTPose.egg-info  sort.py	   train.py	    vit_utils
config.yaml  inference.py	    to_onnx.ipynb  utils_engine.py
datasets     __init__.py	    to_trt.ipynb   vit_models


In [ ]:
# If you get an error from PIL restart environment and rerun this cell to update packages version
# from easy_ViTPose.inference import VitInference
import abc
import time
from typing import Optional
import json
import os
import tqdm

from PIL import Image
import cv2
import numpy as np
import torch

from configs.ViTPose_common import data_cfg
from vit_models.model import ViTPose
from vit_utils.top_down_eval import keypoints_from_heatmaps
from vit_utils.visualization import draw_points_and_skeleton, joints_dict
from vit_utils.inference import pad_image, VideoReader, NumpyEncoder, draw_bboxes
from sort import Sort

try:  # Add bools -> error stack
    import pycuda.driver as cuda  # noqa: [F401]
    import pycuda.autoinit  # noqa: [F401]
    import utils_engine as engine_utils
    import tensorrt as trt
    has_trt = True
except ModuleNotFoundError:
    pass

try:
    import onnxruntime
    has_onnx = True
except ModuleNotFoundError:
    pass

__all__ = ['VitInference']
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]


class VitInference:
    """
    Class for performing inference using ViTPose models with YOLOv5 human detection detection and SORT tracking.

    Args:
        model (str): Path to the ViT model file (.pth, .onnx, .engine).
        yolo_name (str): Name of the YOLOv5 model to load.
        model_name (str, optional): Name of the ViT model architecture to use. Valid values are 's', 'b', 'l', 'h'.
                                    Defaults to None, is necessary when using .pth checkpoints.
        yolo_size (int, optional): Size of the input image for YOLOv5 model. Defaults to 320.
        device (str, optional): Device to use for inference. Defaults to 'cuda' if available, else 'cpu'.
        is_video (bool, optional): Flag indicating if the input is video. Defaults to False.
        single_pose (bool, optional): Flag indicating if the video (on images this flag has no effect) will contain a single pose.
                                      In this case the SORT tracker is not used (increasing performance) but people id tracking
                                      won't be consistent among frames.
        yolo_step (int, optional): The tracker can be used to predict the bboxes instead of yolo for performance,
                                   this flag specifies how often yolo is applied (e.g. 1 applies yolo every frame).
                                   This does not have any effect when is_video is False.
    """

    def __init__(self, model: str,
                 yolo_name: str,
                 model_name: Optional[str] = None,
                 yolo_size: Optional[int] = 320,
                 device: Optional[str] = None,
                 is_video: Optional[bool] = False,
                 single_pose: Optional[bool] = False,
                 yolo_step: Optional[int] = 1):
        assert os.path.isfile(model), f'The model file {model} does not exist'
        assert os.path.isfile(yolo_name), f'The YOLOv5 model {yolo_name} does not exist'

        # Device priority is cuda / mps / cpu
        if device is None:
            if torch.cuda.is_available():
                device = 'cuda'
            elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
                device = 'mps'
            else:
                device = 'cpu'

        self.device = torch.device(device)
        self.yolo = torch.hub.load("ultralytics/yolov5", "custom", yolo_name)
        self.yolo.to(self.device)
        self.yolo.classes = [0]
        self.yolo_size = yolo_size
        self.yolo_step = yolo_step
        self.is_video = is_video
        self.single_pose = single_pose
        self.reset()

        # State saving during inference
        self.save_state = True  # Can be disabled manually
        self._img = None
        self._yolo_res = None
        self._tracker_res = None
        self._keypoints = None

        # Use extension to decide which kind of model has been loaded
        use_onnx = model.endswith('.onnx')
        use_trt = model.endswith('.engine')

        assert model_name in [None, 's', 'b', 'l', 'h'], \
            f'The model name {model_name} is not valid'

        # onnx / trt models do not require model_cfg specification
        if model_name is None:
            assert use_onnx or use_trt, \
                'Specify the model_name if not using onnx / trt'
        else:
            if model_name == 's':
                from configs.ViTPose_small_coco_256x192 import model as model_cfg
            elif model_name == 'b':
                from configs.ViTPose_base_coco_256x192 import model as model_cfg
            elif model_name == 'l':
                from configs.ViTPose_large_coco_256x192 import model as model_cfg
            elif model_name == 'h':
                from configs.ViTPose_huge_coco_256x192 import model as model_cfg

        self.target_size = data_cfg['image_size']
        if use_onnx:
            self._ort_session = onnxruntime.InferenceSession(model,
                                                             providers=['CUDAExecutionProvider',
                                                                        'CPUExecutionProvider'])
            inf_fn = self._inference_onnx
        elif use_trt:
            logger = trt.Logger(trt.Logger.ERROR)
            trt_runtime = trt.Runtime(logger)
            trt_engine = engine_utils.load_engine(trt_runtime, model)

            # This allocates memory for network inputs/outputs on both CPU and GPU
            self._inputs, self._outputs, self._bindings, self._stream = \
                engine_utils.allocate_buffers(trt_engine)
            # Execution context is needed for inference
            self._context = trt_engine.create_execution_context()
            inf_fn = self._inference_trt
        else:
            self._vit_pose = ViTPose(model_cfg)
            self._vit_pose.eval()

            ckpt = torch.load(model, map_location='cpu')
            if 'state_dict' in ckpt:
                self._vit_pose.load_state_dict(ckpt['state_dict'])
            else:
                self._vit_pose.load_state_dict(ckpt)
            self._vit_pose.to(device)
            inf_fn = self._inference_torch

        # Override inference with selected engine
        self._inference = inf_fn

    def reset(self):
        """
        Reset the inference class to be ready for a new video.
        This will reset the internal counter of frames, on videos
        this is necessary to reset the tracker.
        """
        min_hits = 3 if self.yolo_step == 1 else 1
        use_tracker = self.is_video and not self.single_pose
        self.tracker = Sort(max_age=self.yolo_step,
                            min_hits=min_hits,
                            iou_threshold=0.3) if use_tracker else None  # TODO: Params
        self.frame_counter = 0

    @classmethod
    def postprocess(cls, heatmaps, org_w, org_h):
        """
        Postprocess the heatmaps to obtain keypoints and their probabilities.

        Args:
            heatmaps (ndarray): Heatmap predictions from the model.
            org_w (int): Original width of the image.
            org_h (int): Original height of the image.

        Returns:
            ndarray: Processed keypoints with probabilities.
        """
        points, prob = keypoints_from_heatmaps(heatmaps=heatmaps,
                                               center=np.array([[org_w // 2,
                                                                 org_h // 2]]),
                                               scale=np.array([[org_w, org_h]]),
                                               unbiased=True, use_udp=True)
        return np.concatenate([points[:, :, ::-1], prob], axis=2)

    @abc.abstractmethod
    def _inference(img: np.ndarray) -> np.ndarray:
        """
        Abstract method for performing inference on an image.
        It is overloaded by each inference engine.

        Args:
            img (ndarray): Input image for inference.

        Returns:
            ndarray: Inference results.
        """
        raise NotImplementedError

    def inference(self, img: np.ndarray) -> np.ndarray:
        """
        Perform inference on the input image.

        Args:
            img (ndarray): Input image for inference in RGB format.

        Returns:
            ndarray: Inference results.
        """

        # First use YOLOv5 for detection
        res_pd = np.empty((0, 5))
        results = None
        if (self.tracker is None or
           (self.frame_counter % self.yolo_step == 0 or self.frame_counter < 3)):
            results = self.yolo(img, size=self.yolo_size)
            res_pd = np.array([r[:5].tolist() for r in  # TODO: Confidence threshold
                               results.pandas().xyxy[0].to_numpy() if r[4] > 0.35]).reshape((-1, 5))
        self.frame_counter += 1

        frame_keypoints = {}
        ids = None
        if self.tracker is not None:
            res_pd = self.tracker.update(res_pd)
            ids = res_pd[:, 5].astype(int).tolist()

        # Prepare boxes for inference
        bboxes = res_pd[:, :4].round().astype(int)
        scores = res_pd[:, 4].tolist()
        pad_bbox = 10

        if ids is None:
            ids = range(len(bboxes))

        for bbox, id in zip(bboxes, ids):
            # TODO: Slightly bigger bbox
            bbox[[0, 2]] = np.clip(bbox[[0, 2]] + [-pad_bbox, pad_bbox], 0, img.shape[1])
            bbox[[1, 3]] = np.clip(bbox[[1, 3]] + [-pad_bbox, pad_bbox], 0, img.shape[0])

            # Crop image and pad to 3/4 aspect ratio
            img_inf = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            img_inf, (left_pad, top_pad) = pad_image(img_inf, 3 / 4)

            keypoints = self._inference(img_inf)[0]
            # Transform keypoints to original image
            keypoints[:, :2] += bbox[:2][::-1] - [top_pad, left_pad]
            frame_keypoints[id] = keypoints

        if self.save_state:
            self._img = img
            self._yolo_res = results
            self._tracker_res = (bboxes, ids, scores)
            self._keypoints = frame_keypoints

        return frame_keypoints

    def draw(self, show_yolo=True, show_raw_yolo=False, confidence_threshold=0.5):
        """
        Draw keypoints and bounding boxes on the image.

        Args:
            show_yolo (bool, optional): Whether to show YOLOv5 bounding boxes. Default is True.
            show_raw_yolo (bool, optional): Whether to show raw YOLOv5 bounding boxes. Default is False.

        Returns:
            ndarray: Image with keypoints and bounding boxes drawn.
        """
        img = self._img.copy()
        bboxes, ids, scores = self._tracker_res

        if self._yolo_res is not None and (show_raw_yolo or (self.tracker is None and show_yolo)):
            img = np.array(self._yolo_res.render())[0]

        if show_yolo and self.tracker is not None:
            img = draw_bboxes(img, bboxes, ids, scores)

        img = np.array(img)[..., ::-1]  # RGB to BGR for cv2 modules
        for idx, k in self._keypoints.items():
            img = draw_points_and_skeleton(img.copy(), k,
                                           joints_dict()['coco']['skeleton'],
                                           person_index=idx,
                                           points_color_palette='gist_rainbow',
                                           skeleton_color_palette='jet',
                                           points_palette_samples=10,
                                           confidence_threshold=confidence_threshold)
        return img[..., ::-1], self._keypoints  # Return RGB as original

    def pre_img(self, img):
        org_h, org_w = img.shape[:2]
        img_input = cv2.resize(img, self.target_size, interpolation=cv2.INTER_LINEAR) / 255
        img_input = ((img_input - MEAN) / STD).transpose(2, 0, 1)[None].astype(np.float32)
        return img_input, org_h, org_w

    @torch.no_grad()
    def _inference_torch(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)
        img_input = torch.from_numpy(img_input).to(self.device)

        # Feed to model
        heatmaps = self._vit_pose(img_input).detach().cpu().numpy()
        return self.postprocess(heatmaps, org_w, org_h)

    def _inference_onnx(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)

        # Feed to model
        ort_inputs = {self._ort_session.get_inputs()[0].name: img_input}
        heatmaps = self._ort_session.run(None, ort_inputs)[0]
        return self.postprocess(heatmaps, org_w, org_h)

    def _inference_trt(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)

        # Copy the data to appropriate memory
        np.copyto(self._inputs[0].host, img_input.ravel())
        heatmaps = engine_utils.do_inference(context=self._context,
                                             bindings=self._bindings,
                                             inputs=self._inputs,
                                             outputs=self._outputs,
                                             stream=self._stream)[0]

        # Reshape to output size
        heatmaps = heatmaps.reshape(1, 25, img_input.shape[2] // 4, img_input.shape[3] // 4)
        return self.postprocess(heatmaps, org_w, org_h)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VitInference(model_path, yolo_path, MODEL_SIZE, yolo_size=320, is_video=False, device = device)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-8-25 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Loading /root/.cache/huggingface/hub/models--JunkyByte--easy_ViTPose/snapshots/2e599f9067ef175c7e270bafca586d1cf8d3f9df/yolov5/yolov5n.onnx for ONNX Runtime inference..

[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/Pillow-9.4.0.dist-info/METADATA'


Adding AutoShape... 


# Todo
## 0. 접근 방향
- 먼저 VitPose로 각 이미지에 대한 keypoint를 추출한다.
- 그 후 해당 keypoint들을 사용하여 어떤 동작인지 classification task를 진행한다.
- 이를 위해 먼저 모든 이미지 데이터에 대해 keypoint를 추출한 후 해당 데이터들을 사용하여 classification 진행(1시간 가량 소요)

## 1. ViTPose
- model.inference시 ndarray 타입을 input으로 줘야해서 이때는 pytorch 데이터로더 사용 x
- 총 3000여장 데이터, keypoint추출에 약 1시간여 소요
- 추출한 keypoint와 label을 .npy(numpy) 파일로 저장하여 향후 classificiation에 집중할 수 있도록 시도

## 2. Classification
- 1. RandomForest : 25개 관절의 y,x 좌표를 가지는 정형 데이터로 취급하여 randomforest 모델 사용 (acc 0.5가량)
- 2. Basic MLP : 간단한 MLP를 사용하여 딥러닝 시도(acc 0.01 가량)
- 3. RNN? CNN?

- normalization
- 좌표 상대화


# Dataset
- model.inference가 torch.tensor가 아닌 ndarray type을 input으로 받아야 해서 dataloader 사용 X

In [ ]:
path =  '/content/gdrive/MyDrive/kubig_pose/원천데이터'

In [ ]:
import os
from glob import glob

In [ ]:
folders = os.listdir(path)

In [ ]:
images = []
for i in folders:
  folder_name = i.split('(')[0].replace(' ', '')
  for j in glob(path + f'/{i}/*'):
    images.append([folder_name, j])
images[:5]

[['앞서고안막기',
  '/content/gdrive/MyDrive/kubig_pose/원천데이터/앞서고 안막기/B-005-A036-F-B2006-S-20211126-01-01-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/kubig_pose/원천데이터/앞서고 안막기/B-005-A036-F-B2006-S-20211126-01-07-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/kubig_pose/원천데이터/앞서고 안막기/B-005-A035-M-B2006-S-20211109-01-01-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/kubig_pose/원천데이터/앞서고 안막기/B-005-A035-M-B2006-S-20211109-01-07-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/kubig_pose/원천데이터/앞서고 안막기/B-005-A037-M-C1999-S-20211104-01-01-E01.jpg']]

In [ ]:
import pandas as pd
data = pd.DataFrame(images)
data.columns = ['label','path']

In [ ]:
data.label.value_counts()

앞굽이하고지르기                      446
앞서고지르기                           290
앞굽이하고안막기                    162
뒷굽이하고손날거들어바깥막기     101
범서고바탕손안막기                  80
뒷굽이하고바깥막기                   80
앞굽이하고등주먹앞치기              80
앞굽이하고아래막기                    80
옆차기                                  74
주춤서고손날옆막기                  64
주춤서고팔꿈치표적치기               64
옆서고메주먹내려치기                   61
앞굽이하고헤쳐막기                    60
앞서고손날안치기                     60
앞서고아래막기                         60
앞서고등주먹바깥치기                 60
앞굽이하고팔꿈치거들어돌려치기     60
앞굽이하고팔꿈치표적치기            60
주춤서고옆지르기                       53
두발당성차기                          44
앞차기                                  42
범서고바깥막기                        42
돌려차기                       

In [ ]:
len(data)

3118

In [ ]:
label_dict = {i : idx for idx, i in enumerate(data.label.unique())}

In [ ]:
data['label'] = data['label'].map(label_dict)

In [ ]:
# import torch
# import cv2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# transform = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     # ToTensorV2(),
# ])

# class CustomImageDataset(torch.utils.data.Dataset):
#     def __init__(self, label, dir, tf):
#           self.img_labels = label
#           self.img_dir = dir
#           self.transform = tf

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         image = cv2.cvtColor(cv2.imread(self.img_dir.iloc[idx]), cv2.COLOR_BGR2RGB)
#         label = self.img_labels.iloc[idx]
#         transformed = self.transform(image=image)
#         image = transformed['image']
#         return image, label

In [ ]:
# TaekwonDataset = CustomImageDataset(data['label'], data['path'], transform)

In [ ]:
# TaekwonDataloader = torch.utils.data.DataLoader(TaekwonDataset, shuffle=True)

# From TaekwonDataset infer keypoint

In [ ]:
from tqdm.auto import tqdm
keypoints = []
labels = []

for i in tqdm(zip(data['label'], data['path'])):
  label = i[0]
  keypoint = model.inference(cv2.cvtColor(cv2.imread(i[1]), cv2.COLOR_BGR2RGB))
  keypoints.append(keypoint)
  labels.append(label)

0it [00:00, ?it/s]

In [ ]:
np.save('/content/gdrive/MyDrive/kubig_pose/keypoints', np.array(keypoints))

In [ ]:
np.save('/content/gdrive/MyDrive/kubig_pose/labels', np.array(labels))

# From keypoint to classification

# 1. RandomForest(ML) -> with tabular data

In [ ]:
def keypoint_flatten(keypoints, label):
  columns = []

  for i in range(25):
    columns.extend([f'y{i}', f'x{i}'])
  columns.append('label')
  ml_data = pd.DataFrame(columns = columns)


  for idx, keypoint in enumerate(keypoints):
    flattend_data = []

    try :
      for i in keypoint[0]:
        flattend_data.append(i[0])
        flattend_data.append(i[1])
      flattend_data.append(label[idx])
      ml_data.loc[idx] = flattend_data
    except :
      pass
  return ml_data

In [ ]:
df = keypoint_flatten(keypoints, labels)
x = df.drop(columns = 'label')
y = df['label']

In [ ]:
df[:5]

,y0,x0,y1,x1,y2,...,y23,x23,y24,x24,label
0,205.177368,966.683228,194.080811,979.200867,193.277130,...,619.279541,932.659607,601.986206,948.716980,0.0
1,211.253021,567.365601,205.044968,570.523926,204.725693,...,506.744324,644.375732,505.097046,678.189026,0.0
2,149.869995,943.557556,140.361084,953.411743,140.619995,...,520.015259,914.174866,504.355469,924.018311,0.0
3,179.628754,718.676331,173.180954,724.367920,172.443756,...,491.723633,779.317139,493.803284,818.889893,0.0
4,100.417236,921.562256,92.367981,929.884583,92.681030,...,427.179443,890.333801,413.665283,904.144775,0.0


In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, stratify = y, test_size = 0.15, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(train_x, train_y)

RandomForestClassifier()

In [ ]:
model.score(valid_x, valid_y)

0.6173913043478261

# 2. Basic MLP

In [ ]:
from torch import nn
import torch.nn.functional as F

class basic_mlp(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(50, 64)
    self.fc2 = nn.Linear(64,128)
    self.fc3 = nn.Linear(128, 47)
    self.dropout = nn.Dropout(p = 0.3)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = F.softmax(x)
    return x

In [ ]:
def from_keypoint_to_input(keypoint):
  input_data = []
  for i in keypoint:
    input_data.append(i[:-1])
  return np.array(input_data)

def keypoint_mlp(keypoints, labels):
  flattend_data = []
  label = []
  for idx, keypoint in enumerate(keypoints):
    try :
        flattend_data.append(torch.tensor(from_keypoint_to_input(keypoint[0]).flatten()))
        label.append(labels[idx])
    except :
      pass
  return flattend_data, label

In [ ]:
x, y = keypoint_mlp(keypoints, labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, stratify = y, test_size = 0.15, random_state = 42)

In [ ]:
model = basic_mlp()
model.to(device)

basic_mlp(
  (fc1): Linear(in_features=50, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=47, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

In [ ]:
class CustomKeypointsDataset(torch.utils.data.Dataset):
    def __init__(self, keypoints, labels):
        self.keypoints = keypoints
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        keypoint = self.keypoints[idx]
        label = self.labels[idx]
        return keypoint, label

In [ ]:
SkeletonTrainDataset = CustomKeypointsDataset(train_x, train_y)
SkeletonValidDataset = CustomKeypointsDataset(valid_x, valid_y)

In [ ]:
TrainDataloader = torch.utils.data.DataLoader(SkeletonTrainDataset, batch_size = 128, shuffle = True)
ValidDataloader = torch.utils.data.DataLoader(SkeletonValidDataset, batch_size = 128,shuffle = False)

In [ ]:
for epoch in range(50):
  train_loss = []
  train_acc = []
  valid_loss = []
  valid_acc = []

  for skeleton, cls in iter(TrainDataloader):
    optimizer.zero_grad()

    skeleton, cls = skeleton.to(device), cls.to(device)

    output = model(skeleton)
    loss = criterion(output, cls)
    train_loss.append(loss.item())
    train_acc.append(sum(torch.max(output, dim=1)[1] == cls) / skeleton.shape[0])
    loss.backward()
    optimizer.step()

  with torch.no_grad():
    for skeleton, cls in iter(ValidDataloader):
      skeleton, cls = skeleton.to(device), cls.to(device)
      output = model(skeleton)
      loss = criterion(output, cls)
      valid_loss.append(loss.item())
      valid_acc.append(sum(torch.max(output, dim=1)[1] == cls) / skeleton.shape[0])

  print(f'epoch {epoch+1} -- train_loss : {sum(train_loss[-len(TrainDataloader):]) / len(TrainDataloader)} \
  train_acc : {sum(train_acc[-len(TrainDataloader):]) / len(TrainDataloader)}\
   valid_loss : {sum(valid_loss[-len(ValidDataloader):]) / len(ValidDataloader)} \
  valid_acc : {sum(valid_acc[-len(ValidDataloader):]) / len(ValidDataloader)}')

epoch 1 -- train_loss : 3.8691863332475935   train_acc : 0.01666019670665264   valid_loss : 3.8684685230255127   valid_acc : 0.017578125
epoch 2 -- train_loss : 3.867060740788778   train_acc : 0.01897321455180645   valid_loss : 3.8684741854667664   valid_acc : 0.017578125
epoch 3 -- train_loss : 3.866414251781645   train_acc : 0.01963638700544834   valid_loss : 3.8664978742599487   valid_acc : 0.01953125
epoch 4 -- train_loss : 3.8664140701293945   train_acc : 0.01963638700544834   valid_loss : 3.868476390838623   valid_acc : 0.017578125
epoch 5 -- train_loss : 3.8670782361711775   train_acc : 0.01897321455180645   valid_loss : 3.8684768080711365   valid_acc : 0.017578125
epoch 6 -- train_loss : 3.865269865308489   train_acc : 0.020962731912732124   valid_loss : 3.868476390838623   valid_acc : 0.017578125
epoch 7 -- train_loss : 3.866418270837693   train_acc : 0.01963638700544834   valid_loss : 3.868476390838623   valid_acc : 0.017578125
epoch 8 -- train_loss : 3.867081437792097   trai